In [ ]:
pip install pygad 

     |████████████████████████████████| 53 kB 1.4 MB/s 


In [ ]:
import pygad
import numpy as np
import scipy

In [ ]:
X= np.array([[1,0,0,0]]).T
Y = np.array([[0,1,0,0],[0,0,1,0],[0,0,1,0],[0,0,0,1]]).T

In [ ]:
word = "hello"
chars = set(word)
int2char = dict(enumerate(chars))
char2int = {char:idx for idx,char in int2char.items()}
print(int2char,char2int)

{0: 'o', 1: 'e', 2: 'h', 3: 'l'} {'o': 0, 'e': 1, 'h': 2, 'l': 3}


In [ ]:
inp,out = [],[]
inp = list(word[:-1])
out = list(word[1:])
print(inp,out)
for i in range(len(word)-1):
  inp[i] = char2int[inp[i]]
  out[i] = char2int[out[i]]
print(inp,out)

['h', 'e', 'l', 'l'] ['e', 'l', 'l', 'o']
[2, 1, 3, 3] [1, 3, 3, 0]


In [ ]:
def one_hot_encoder(vector,seq_length,dict_size):
  features = np.zeros((seq_length,dict_size))
  for i in range(seq_length):
    features[i,vector[i]] = 1
  return features
seq_length = len(inp)
dict_size = len(char2int)
print(one_hot_encoder(inp,seq_length,dict_size))

[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [ ]:
def softmax(x): 
    return np.exp(x)/sum(np.exp(x))

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))


In [ ]:
def crossEntropy(X,Y):
    return -np.sum(np.log(X)*Y)

In [ ]:
def crossEntropy(X,Y):
    return -np.sum(np.log(X)*Y)

In [ ]:
def LSTM_GA(solution, solution_idx):
    Wf = solution[0:21].reshape(3,7)
    Wi = solution[21:42].reshape(3,7)
    Wc = solution[42:63].reshape(3,7)
    Wo = solution[63:84].reshape(3,7)
    bf = solution[84:87].reshape(3,1)
    bi = solution[87:90].reshape(3,1)
    bc = solution[90:93].reshape(3,1)
    bo = solution[93:96].reshape(3,1)
    Why = solution[96:108].reshape(4,3)
    by = solution[108:112].reshape(4,1)
    
    
    h_s = np.zeros((3,1))
    y_s = X
    C_s = np.zeros((3,1))

    for t in range(X.shape[0]):
        x_t = y_s[:,-1].reshape(4,1)
        z_t = np.vstack((x_t,h_s[:,-1].reshape(3,1)))  ## x_t + h_(t-1)
        
        f_t = sigmoid(np.dot(Wf,z_t)+bf)
        i_t = sigmoid(np.dot(Wi,z_t)+bi)
        C_t1 = np.tanh(np.dot(Wc,z_t)+bi) # C mu t
        C_t = f_t*C_s[:,-1].reshape(3,1) +i_t*C_t1
        o_t = sigmoid(np.dot(Wo,z_t)+bo)
        h_t = o_t*np.tanh(C_t)
        y_t = softmax(np.dot(Why, h_t) + by) 
        
        h_s = np.hstack((h_s,h_t)) 
        y_s = np.hstack((y_s,y_t))
        C_s = np.hstack((C_s,C_t))
    h_s = h_s[:,1:]
    y_s = y_s[:,1:]
    return 1/np.sum(crossEntropy(y_s,Y))

In [ ]:
def LSTM(solution, solution_idx,X,Y):
    Wf = solution[0:21].reshape(3,7)
    Wi = solution[21:42].reshape(3,7)
    Wc = solution[42:63].reshape(3,7)
    Wo = solution[63:84].reshape(3,7)
    bf = solution[84:87].reshape(3,1)
    bi = solution[87:90].reshape(3,1)
    bc = solution[90:93].reshape(3,1)
    bo = solution[93:96].reshape(3,1)
    Why = solution[96:108].reshape(4,3)
    by = solution[108:112].reshape(4,1)
    
    
    h_s = np.zeros((3,1))
    y_s = X
    C_s = np.zeros((3,1))

    for t in range(X.shape[0]):
        x_t = y_s[:,-1].reshape(4,1)
        z_t = np.vstack((x_t,h_s[:,-1].reshape(3,1)))  ## x_t + h_(t-1)
        
        f_t = sigmoid(np.dot(Wf,z_t)+bf)
        i_t = sigmoid(np.dot(Wi,z_t)+bi)
        C_t1 = np.tanh(np.dot(Wc,z_t)+bi) # C mu t
        C_t = f_t*C_s[:,-1].reshape(3,1) +i_t*C_t1
        o_t = sigmoid(np.dot(Wo,z_t)+bo)
        h_t = o_t*np.tanh(C_t)
        y_t = softmax(np.dot(Why, h_t) + by) 
        
        h_s = np.hstack((h_s,h_t)) 
        y_s = np.hstack((y_s,y_t))
        C_s = np.hstack((C_s,C_t))
    h_s = h_s[:,1:]
    y_s = y_s[:,1:]
    return h_s,y_s,crossEntropy(y_s,Y)

In [ ]:
fitness_function = LSTM_GA
num_generations = 100
num_parents_mating = 10
sol_per_pop = 20
num_genes = 116

init_range_low = -2
init_range_high = 5

parent_selection_type = "sss"
keep_parents = 1

crossover_type ="single_point"

mutation_type = "random"
mutation_percent_genes = 20


ga_instance = pygad.GA(
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes)

ga_instance.run()

solution, solution_fitness, solution_idx = ga_instance.best_solution()

print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))




Fitness value of the best solution = 4795.422611964595


In [ ]:
h,y,Loss =LSTM(solution,solution,X,Y)
print("Ket qua thu duoc y = \n {} \n Ket qua sau khi lam tron 3 chu so y= \n {}. \n Loss= {} ".format(y,np.round(y,decimals=3),Loss))

Ket qua thu duoc y = 
 [[3.92802140e-05 4.46306774e-09 7.09707238e-08 6.55187401e-06]
 [9.99856985e-01 2.18921955e-05 1.13958137e-06 1.56525284e-05]
 [7.09024364e-05 9.99976796e-01 9.99994536e-01 1.46341881e-05]
 [3.28318884e-05 1.30729093e-06 4.25337319e-06 9.99963161e-01]] 
 Ket qua sau khi lam tron 3 chu so y= 
 [[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 1. 0.]
 [0. 0. 0. 1.]]. 
 Loss= 0.00020853219432735644 


In [ ]:
Wf = solution[0:21].reshape(3,7)
Wi = solution[21:42].reshape(3,7)
Wc = solution[42:63].reshape(3,7)
Wo = solution[63:84].reshape(3,7)
bf = solution[84:87].reshape(3,1)
bi = solution[87:90].reshape(3,1)
bc = solution[90:93].reshape(3,1)
bo = solution[93:96].reshape(3,1)
Why = solution[96:108].reshape(4,3)
by = solution[108:112].reshape(4,1)

In [ ]:
print("Wf:\n {} ".format(np.round(Wf,decimals=3)))
print("Wi:\n {} ".format(np.round(Wi,decimals=3)))
print("Wc:\n {} ".format(np.round(Wc,decimals=3)))
print("Wo:\n {} ".format(np.round(Wo,decimals=3)))
print("Why:\n {} ".format(np.round(Why,decimals=3)))

Wf:
 [[-3.231 -6.098  1.397  1.67   0.536 -5.929 -2.829]
 [ 3.944  3.435 -0.399  1.947 -0.251  5.454  1.956]
 [ 1.074  0.791  3.521  4.603  6.056  1.949  4.359]] 
Wi:
 [[ 3.687  0.746  1.894 -1.827 -0.665  0.873  6.061]
 [-0.965  6.552 -1.464  2.026  1.656  2.542  0.807]
 [-1.302  2.947 -1.313  0.12   3.517 -1.76  -2.466]] 
Wc:
 [[-3.32   7.1    3.226  1.212 -3.257 -1.68   3.537]
 [-3.198  6.489 -1.213  5.732  2.398  1.302  0.873]
 [-2.041 -0.708  7.896 -5.708  5.177 -1.351  8.03 ]] 
Wo:
 [[ 3.497  2.813  2.886  4.704 -0.717 -2.886  1.585]
 [ 2.838 -0.613  4.181 -1.331  8.143 -2.068  2.395]
 [ 1.488 -2.089  1.068  1.02  -0.46   6.011  3.402]] 
Why:
 [[ -3.014   5.098  -1.262]
 [-10.003  -4.149   1.157]
 [  8.435  -2.521  -0.446]
 [ -4.109   9.484   7.392]] 


In [ ]:
print("bf:\n {} ".format(np.round(bf,decimals=3)))
print("bi:\n {} ".format(np.round(bi,decimals=3)))
print("bc:\n {} ".format(np.round(bc,decimals=3)))
print("bo:\n {} ".format(np.round(bo,decimals=3)))
print("by:\n {} ".format(np.round(by,decimals=3)))

bf:
 [[-0.36 ]
 [ 3.879]
 [ 7.573]] 
bi:
 [[-0.028]
 [-0.015]
 [ 2.714]] 
bc:
 [[2.868]
 [1.87 ]
 [2.517]] 
bo:
 [[ 3.108]
 [-3.82 ]
 [-2.78 ]] 
by:
 [[-1.322]
 [ 5.4  ]
 [ 8.9  ]
 [ 0.04 ]] 
